# 需要的包

In [1]:
# Load in our libraries
import pandas as pd
import numpy as np
import re
import sklearn
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import datetime
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

import warnings
warnings.filterwarnings('ignore')

# Going to use these 5 base models for the stacking
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, ExtraTreesClassifier)
from sklearn.svm import SVC
from sklearn.cross_validation import KFold
pd.set_option('display.float_format', lambda x: '%.3f' % x)
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" #所有的变量信息都可以打印出来
pd.options.display.max_columns = None

/Users/Alan/anaconda2/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


# 数据处理


## 数据读取

In [44]:
types={'CUPS_CARD_IN': str,'SPEC_DISC_LVL': str,'TRANS_CHNL': str,'SPEC_DISC_TP': str,'CARD_BIN': str,'MCHNT_TP': str}
train = pd.read_csv('./data2/trans_dtl_train1.csv',dtype=types)
test  = pd.read_csv('./data2/trans_dtl_test1.csv',dtype=types)
target = pd.read_csv('./data2/bind_cards1.csv')
target.columns = ['encrypt_card_no']
target['target'] = '1'
full_data = [train, test]
all =pd.concat(full_data)
train.info()
train.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312647 entries, 0 to 312646
Data columns (total 32 columns):
SETTLE_DT           312647 non-null int64
encrypt_mchnt_cd    312647 non-null object
TERM_ID             310599 non-null object
MCHNT_TP            311750 non-null object
CONN_IN             312647 non-null int64
LOC_TRANS_TM        312647 non-null int64
STI_TAKEOUT_IN      312647 non-null int64
TRANS_ST            312647 non-null int64
CARD_BIN            312642 non-null object
CARD_PROD           312647 non-null int64
CARD_ATTR           312647 non-null int64
CARD_MEDIA          312647 non-null int64
CARD_BRAND          312647 non-null int64
CARD_RANK           312647 non-null int64
ISS_INS_ID_CD       312647 non-null int64
ACQ_INS_ID_CD       312647 non-null int64
TRANS_ID            312647 non-null object
TRANS_CHNL          312642 non-null object
TRANS_AT            312647 non-null int64
RESP_CD1            312311 non-null object
RESP_CD3            311665 non-null object

,SETTLE_DT,encrypt_mchnt_cd,TERM_ID,MCHNT_TP,CONN_IN,LOC_TRANS_TM,STI_TAKEOUT_IN,TRANS_ST,CARD_BIN,CARD_PROD,CARD_ATTR,CARD_MEDIA,CARD_BRAND,CARD_RANK,ISS_INS_ID_CD,ACQ_INS_ID_CD,TRANS_ID,TRANS_CHNL,TRANS_AT,RESP_CD1,RESP_CD3,RESP_CD4,SPEC_DISC_TP,SPEC_DISC_LVL,encrypt_card_no,CUPS_CARD_IN,TRANS_MD,HCE_PROD_NM,TOUCH_TP,HCE_PROD_IN,NOPWD_PETTY_IN,WHITE_MCHNT_IN
0,20180612,42fc2d6c76d2048c9aded5dbfcd1a9e2,12042365,8220,1,72535,1,1,19620516,0,5,3,1,1,801020000,801035800,I22,03,400,00,00,00,00,0,7ccca9ac2b0fa257c3f78e4a0a30730c,2,1,0,2,0,0,1
1,20180603,f7fcc1f8027ff424327f7f6a2a2da256,01080209,4816,1,175047,1,1,1962156831,3,1,4,1,2,801046500,803101131,S31,07,95000,00,00,00,03,0,82c32cbb47703f4c3e6fd4585bbab5ce,2,2,0,99,0,0,0
2,20180511,e552f7d0516b669e04cbecc16b0b6640,45880776,7011,0,73149,1,1,16628268,1,2,3,1,2,801030001,801046100,S20,03,46000,00,00,00,00,0,edd137b31c3112c69c5e9f7d366dd81d,2,2,0,2,0,0,0
3,20180602,047e9e99f210f924ba30260eab354531,10000001,8398,1,63544,1,1,1962309101,0,1,4,1,3,814293310,800010035,S56,07,486,00,00,00,00,0,55342a18c527b31709f9ee9ba2663c59,2,2,Y,99,0,0,1
4,20180606,42fc2d6c76d2048c9aded5dbfcd1a9e2,12042426,8220,1,124212,1,1,19620516,0,5,3,1,1,801020000,801035800,I22,03,600,00,00,00,00,0,953664ea99b5b2b3fddb08732a358aa7,2,1,0,2,0,0,1


In [3]:
train['MCHNT_TP'].nunique()

282

In [4]:
train['TOUCH_TP'].nunique()
train['CARD_BIN'].nunique()

3

975

## 数据统计

In [133]:
test.shape
train.shape
target['encrypt_card_no'].nunique()
test['encrypt_card_no'].nunique()
train['encrypt_card_no'].nunique()
all['encrypt_card_no'].nunique()

(1026305, 32)

(312647, 32)

1199

35001

11200

46201

## 提取简易特征 


### 数字特征

In [45]:
train['SETTLE_DT1'] =pd.to_datetime(train['SETTLE_DT'], format='%Y%m%d')
train['date_diff'] = (pd.Timestamp('2018-08-03')- train['SETTLE_DT1']).dt.days
# train['TRANS_AT1'] = np.log1p(train['TRANS_AT'] )
train['LOC_TRANS_TM1']  = train['LOC_TRANS_TM'] .apply(lambda s: str(s)[:-4])

In [9]:
train['LOC_TRANS_TM1']

0          7
1         17
2          7
3          6
4         12
5          9
6         12
7          6
8         12
9         17
10        12
11        19
12        18
13        23
14        20
15         8
16        19
17        14
18        18
19        12
20        10
21        21
22        20
23        20
24         9
25         9
26        22
27        10
28        14
29        11
          ..
312617    14
312618    23
312619    18
312620    10
312621    17
312622    21
312623    17
312624     8
312625    11
312626     7
312627    10
312628    10
312629     9
312630    12
312631     8
312632    15
312633     8
312634    18
312635    12
312636     8
312637    10
312638    19
312639    15
312640    12
312641    11
312642    12
312643     7
312644    12
312645    16
312646    16
Name: LOC_TRANS_TM1, Length: 312647, dtype: object

### 类别特征

#### MCC之下，商户类型又分为了3个层次的不同类别

In [46]:
mcc = pd.read_csv('./data2/Para/PARA_DIM_MCC.txt',encoding='gb2312',header=0)
train=pd.merge(train,mcc[['MCHNT_TP','MCC_DESC','MCC_GRP_DESC1','MCC_GRP_DESC2','MCC_GRP_DESC3']]  , on='MCHNT_TP', how='left')

In [47]:
mcc['MCC_GRP_DESC1'].nunique()
mcc['MCC_GRP_DESC2'].nunique()
mcc['MCC_GRP_DESC3'].nunique()
train['MCHNT_TP'].nunique()

30

15

4

282

#### CARD_BRAND 后续将visa，jcb等归为一类。

In [48]:
CARD_BRAND = pd.read_csv('./data2/Para/PARA_CARD_BRAND.txt',encoding='gb2312',header=0)
CARD_BRAND


,CARD_BRAND,CARD_BRAND_NM,REC_UPD_TS
0,0,其它,2008-11-14-13.31.18.000000
1,1,6字头银标,2008-11-14-13.31.18.000000
2,2,6字头非标,2008-11-14-13.31.18.000000
3,3,银联9字头,2008-11-14-13.31.18.000000
4,4,VISA卡,2008-11-14-13.31.18.000000
5,5,MASTER卡,2008-11-14-13.31.18.000000
6,6,JCB,2008-11-14-13.31.18.000000
7,7,美运卡,2008-11-14-13.31.18.000000
8,8,其它卡BIN,2008-11-14-13.31.18.000000


In [49]:
DIM_TRANS = pd.read_csv('./data2/Para/PARA_DIM_TRANS.txt',encoding='gb2312',header=0)
# DIM_TRANS
# train=pd.merge(train,mcc[['MCHNT_TP','MCC_DESC','MCC_GRP_DESC1','MCC_GRP_DESC2','MCC_GRP_DESC3']]  , on='MCHNT_TP', how='left')
DIM_TRANS['TRANS_DESC'].nunique()
DIM_TRANS['TRANS_GRP_DESC1'].nunique()
DIM_TRANS['TRANS_GRP_DESC2'].nunique()
DIM_TRANS['TRANS_GRP_DESC3'].nunique()
train['TRANS_ID'].nunique()
train=pd.merge(train,DIM_TRANS[['TRANS_ID','TRANS_GRP_DESC1','TRANS_GRP_DESC2','TRANS_GRP_DESC3']]  , on='TRANS_ID', how='left')

455

79

21

6

58

In [50]:
DIM_RESP = pd.read_csv('./data2/Para/PARA_DIM_RESP.txt',encoding='gb2312',header=0)
DIM_RESP

,resp_cd,RESP_DESC,rec_crt_ts,rec_upd_ts
0,42,无此账户,06DEC2008:11:04:01.000000,06DEC2008:11:04:01.000000
1,62,受限制的卡,06DEC2008:11:04:01.000000,06DEC2008:11:04:01.000000
2,25,找不到原始交易,06DEC2008:11:04:01.000000,06DEC2008:11:04:01.000000
3,64,原始金额不正确,06DEC2008:11:04:01.000000,06DEC2008:11:04:01.000000
4,97,ATM/POS终端号找不到,06DEC2008:11:04:01.000000,06DEC2008:11:04:01.000000
5,22,怀疑操作有误,06DEC2008:11:04:01.000000,06DEC2008:11:04:01.000000
6,68,收到的回答太迟,06DEC2008:11:04:01.000000,06DEC2008:11:04:01.000000
7,33,过期的卡（没收卡）,06DEC2008:11:04:01.000000,06DEC2008:11:04:01.000000
8,16,批准更新第三磁道,06DEC2008:11:04:01.000000,06DEC2008:11:04:01.000000
9,A8,PIN密钥错,06DEC2008:11:04:01.000000,06DEC2008:11:04:01.000000


In [51]:
train['ISS_INS_ID_CD'].nunique()
train['RESP_CD4'].nunique()

468

44

### 根据卡号进行聚合

In [9]:
train.columns

Index(['SETTLE_DT', 'encrypt_mchnt_cd', 'TERM_ID', 'MCHNT_TP', 'CONN_IN',
       'LOC_TRANS_TM', 'STI_TAKEOUT_IN', 'TRANS_ST', 'CARD_BIN', 'CARD_PROD',
       'CARD_ATTR', 'CARD_MEDIA', 'CARD_BRAND', 'CARD_RANK', 'ISS_INS_ID_CD',
       'ACQ_INS_ID_CD', 'TRANS_ID', 'TRANS_CHNL', 'TRANS_AT', 'RESP_CD1',
       'RESP_CD3', 'RESP_CD4', 'SPEC_DISC_TP', 'SPEC_DISC_LVL',
       'encrypt_card_no', 'CUPS_CARD_IN', 'TRANS_MD', 'HCE_PROD_NM',
       'TOUCH_TP', 'HCE_PROD_IN', 'NOPWD_PETTY_IN', 'WHITE_MCHNT_IN',
       'SETTLE_DT1', 'date_diff', 'TRANS_AT2', 'MCC_DESC', 'MCC_GRP_DESC1',
       'MCC_GRP_DESC2', 'MCC_GRP_DESC3', 'TRANS_GRP_DESC1', 'TRANS_GRP_DESC2',
       'TRANS_GRP_DESC3'],
      dtype='object')

未处理的字段 encrypt_mchnt_cd TERM_ID

In [52]:
types={'CUPS_CARD_IN': str,'SPEC_DISC_LVL': str,'TRANS_CHNL': str,'SPEC_DISC_TP': str,'CARD_BIN': str,'MCHNT_TP': str}

all_together =(train.groupby(['encrypt_card_no'])
                  .agg({
                      'date_diff': [ 'mean', 'min', 'max'],
                      'TRANS_AT': ['sum', 'mean', 'min', 'max','count'],
                      'HCE_PROD_NM': ['min', 'max', 'nunique'],
                       'MCC_DESC': [  'last','first', 'nunique'],
                      'MCC_GRP_DESC3': [  'last','first', 'nunique'],
                      'MCC_GRP_DESC1': [  'last','first', 'nunique'],
                      'MCC_GRP_DESC2': [  'last','first', 'nunique'],
                      'CONN_IN': ['min', 'nunique'],
                      'LOC_TRANS_TM1': ['min', 'max','nunique','last'],
                      'STI_TAKEOUT_IN': ['max','nunique'],
                      'TRANS_ST': ['min', 'max','last','nunique'],
                      'CARD_PROD': ['last','nunique'],
                      'CARD_ATTR': ['last','nunique'],
                      'CARD_MEDIA': ['last','nunique'],
                      'CARD_BRAND': ['last','nunique'],
                      'CARD_RANK': ['nunique', 'max'],
                      'TRANS_CHNL': [  'last','first', 'nunique'],
                      'TRANS_GRP_DESC1': [  'last','first', 'nunique'],
                      'TRANS_GRP_DESC2': [ 'last','first', 'nunique'],
                      'TRANS_GRP_DESC3': [  'last','first', 'nunique'],
                      'RESP_CD3': [  'last','first', 'nunique'],
                      'SPEC_DISC_TP': [ 'last','first', 'nunique'],
                      'SPEC_DISC_LVL': [  'last','first', 'nunique'],
                      'CUPS_CARD_IN': [  'last', 'nunique'],
                      'TRANS_MD': [  'max','min','last','nunique'],
                      'TOUCH_TP': [ 'first','last', 'nunique'],
                      'HCE_PROD_IN': [  'last','nunique'],
                      'NOPWD_PETTY_IN': [  'last','nunique'],
                      'WHITE_MCHNT_IN': [  'last','nunique']
                  }))

In [53]:
all_together.fillna('MISSING',inplace = True)

### rename column

In [54]:
all_together.columns = ["_".join(x) for x in all_together.columns.ravel()]
all_together=all_together.reset_index(level=['encrypt_card_no'],drop=False)
all_together.head(2)

,encrypt_card_no,date_diff_mean,date_diff_min,date_diff_max,TRANS_AT_sum,TRANS_AT_mean,TRANS_AT_min,TRANS_AT_max,TRANS_AT_count,HCE_PROD_NM_min,HCE_PROD_NM_max,HCE_PROD_NM_nunique,MCC_DESC_last,MCC_DESC_first,MCC_DESC_nunique,MCC_GRP_DESC3_last,MCC_GRP_DESC3_first,MCC_GRP_DESC3_nunique,MCC_GRP_DESC1_last,MCC_GRP_DESC1_first,MCC_GRP_DESC1_nunique,MCC_GRP_DESC2_last,MCC_GRP_DESC2_first,MCC_GRP_DESC2_nunique,CONN_IN_min,CONN_IN_nunique,LOC_TRANS_TM1_min,LOC_TRANS_TM1_max,LOC_TRANS_TM1_nunique,LOC_TRANS_TM1_last,STI_TAKEOUT_IN_max,STI_TAKEOUT_IN_nunique,TRANS_ST_min,TRANS_ST_max,TRANS_ST_last,TRANS_ST_nunique,CARD_PROD_last,CARD_PROD_nunique,CARD_ATTR_last,CARD_ATTR_nunique,CARD_MEDIA_last,CARD_MEDIA_nunique,CARD_BRAND_last,CARD_BRAND_nunique,CARD_RANK_nunique,CARD_RANK_max,TRANS_CHNL_last,TRANS_CHNL_first,TRANS_CHNL_nunique,TRANS_GRP_DESC1_last,TRANS_GRP_DESC1_first,TRANS_GRP_DESC1_nunique,TRANS_GRP_DESC2_last,TRANS_GRP_DESC2_first,TRANS_GRP_DESC2_nunique,TRANS_GRP_DESC3_last,TRANS_GRP_DESC3_first,TRANS_GRP_DESC3_nunique,RESP_CD3_last,RESP_CD3_first,RESP_CD3_nunique,SPEC_DISC_TP_last,SPEC_DISC_TP_first,SPEC_DISC_TP_nunique,SPEC_DISC_LVL_last,SPEC_DISC_LVL_first,SPEC_DISC_LVL_nunique,CUPS_CARD_IN_last,CUPS_CARD_IN_nunique,TRANS_MD_max,TRANS_MD_min,TRANS_MD_last,TRANS_MD_nunique,TOUCH_TP_first,TOUCH_TP_last,TOUCH_TP_nunique,HCE_PROD_IN_last,HCE_PROD_IN_nunique,NOPWD_PETTY_IN_last,NOPWD_PETTY_IN_nunique,WHITE_MCHNT_IN_last,WHITE_MCHNT_IN_nunique
0,00050a15df80a709d3e4973543b27c7a,54.000,2,81,4050743,253171.438,2350,1500000,16,0,0,1,综合性超级市场,成人成衣店,10,02优惠类,01标准类,2,18超市类,09日用百货类,6,06超市加油类,03百货一般类,3,0,2,10,9,7,11,1,2,1,1,1,1,0,1,2,1,3,1,1,1,1,2,03,03,1,消费,消费,1,消费类,消费类,1,常规类交易代码,常规类交易代码,1,00,00,3,00,00,2,0,0,1,2,1,2,2,2,1,1,1,2,0,1,0,2,1,2
1,0007358abae98a7836c3ec09c3b51e61,24.706,2,46,320780,18869.412,0,50000,17,0,0,1,金融机构-自动现金支付,金融机构-自动现金支付,5,01标准类,01标准类,2,14一般票据类,14一般票据类,5,03百货一般类,03百货一般类,4,0,2,11,9,5,11,1,2,1,1,1,1,0,1,1,1,1,2,1,1,1,1,01,01,3,查询,查询,4,查询类,查询类,4,常规类交易代码,常规类交易代码,1,00,00,3,00,00,2,0,0,1,2,1,2,2,2,1,99,99,1,0,1,0,1,0,2


## 打标签


###  直接打标签

根据bind_cards1.csv打标签，绑定云闪付的为1，不绑定的为0

In [55]:
train_v=pd.merge(all_together, target, on='encrypt_card_no', how='outer')
train_v =train_v.fillna(value=0)
train_v.shape

(11200, 83)

In [57]:
y = train_v[['target']]
y['target'] = y['target'].astype(str).astype(int)

In [58]:
train_v = train_v.drop(columns=['encrypt_card_no', 'target'])

In [59]:
train_dummy = pd.get_dummies(train_v)

In [60]:
def print_output(y,y_pred):
    precision = metrics.precision_score(y, y_pred)
    recall = metrics.recall_score(y, y_pred)
    F1 = 2*precision*recall/(precision + recall)

    print("Precision : %.4g" % precision)
    print("Recall: %f" % recall)
    print("F1: %f" % F1)
    return precision,recall,F1

In [61]:
result = main(5, 0.3, train_dummy, y['target'], 0)

Precision : 0.9833
Recall: 0.821229
F1: 0.894977
Precision : 0.9839
Recall: 0.868946
F1: 0.922844
Precision : 0.9778
Recall: 0.841530
F1: 0.904552
Precision : 0.9903
Recall: 0.825269
F1: 0.900293
Precision : 0.974
Recall: 0.859599
F1: 0.913242


0-无产品          1-Apple Pay       2-HCE       3-三星pay mst  4-三星pay ic  5-线上大商户          6-华为 

7-小米          8-中兴          Y-主扫         Z-被扫

In [126]:
x = np.nan_to_num(train_dummy)

In [108]:
x

array([[54.        ,  2.        , 81.        , ...,  0.        ,
         1.        ,  0.        ],
       [24.70588235,  2.        , 46.        , ...,  0.        ,
         1.        ,  0.        ],
       [14.2       ,  1.        , 65.        , ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [39.33333333,  2.        , 70.        , ...,  0.        ,
         1.        ,  0.        ],
       [ 2.        ,  2.        ,  2.        , ...,  0.        ,
         1.        ,  0.        ],
       [19.57142857,  2.        , 35.        , ...,  0.        ,
         1.        ,  0.        ]])

In [109]:
pd.DataFrame(train_dummy).fillna()

ValueError: must specify a fill method or value

In [10]:
train.groupby('HCE_PROD_NM').size()

HCE_PROD_NM
0    305217
1      1951
2        43
4        10
6        85
7         9
Y      4145
Z      1187
dtype: int64

# 结果文件

In [19]:
import pandas as pd
import numpy as np
import math
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import cross_validation, metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [20]:
def standard_data(data):
    scaler = preprocessing.StandardScaler()
    scaler.fit(data)
    data_normorlize = scaler.transform(data)

    return data_normorlize

In [21]:
def train_classifier(X, Y):
    gbm0 = GradientBoostingClassifier(random_state=10)
    gbm0.fit(X,Y)
    return gbm0

In [22]:
def main(n,Test_size,X_total,Y_total,ifnorm):
    result_ar = np.ones((n,3))
    for i in range(n):
        X_train, X_test, y_train, y_test = train_test_split(X_total, Y_total, test_size=Test_size, random_state=i)
        if ifnorm:
            X_train = standard_data(X_train)
            X_test = standard_data(X_test)
        GBDT1 = train_classifier(X_train, y_train)
        y_pred = GBDT1.predict(X_test)
        result_ar[i,:] = print_output(y_test, y_pred)
    return  result_ar

In [ ]:
result = main(5, 0.3, X_total, Y_total, 0)

In [ ]:
a=train.loc[test['HCE_PROD_NM'] != '0'].encrypt_card_no.unique()
s=pd.Series(a)
s.to_csv('2.csv',index=False)

In [137]:
a=test.loc[test['HCE_PROD_NM'] != '0'].encrypt_card_no.unique()
pd.DataFrame(a).to_csv("1.csv",header=False,index=False)